In [ ]:
include("MiniCollectiveSpins.jl")
using PyPlot
using Statistics
using JLD2
using OrdinaryDiffEq
import PhysicalConstants.CODATA2018: c_0
using Unitful
using ProgressMeter
using NonlinearSolve
using SteadyStateDiffEq 
using BenchmarkTools
using Libdl

In [ ]:
""" Prepare the initial vector u0 for MPC (fcounter=1) or MF (fcounter=2) """
function u0_CFunction(phi_array, theta_array, op_list, fcounter)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            if fcounter == 2
                print("Not a meanfield differential equation!")
            else
                for op in op_list[i]
                    j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                    if parse(Int, string(op)[1:2]) == 22
                        u0[i] *= cos(theta_array[j]/2)^2
                    elseif parse(Int, string(op)[1:2]) == 21
                        u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                    elseif parse(Int, string(op)[1:2]) == 12
                        u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                    else
                        println(op)
                    end
                end
            end
        end
    end
    return u0
end



""" Create a random distribution, save it, computes the corresponding parameters an return the stationnary state. 
If compute_t_evolution, compute the whole evolution, else only the stationnary state. """
function solve_random_distrib(chunk, op_list, N, n, d0_lb, fcounter)
    popup_t, sol_t, t_sol = [], [], []

    for i in chunk
        # Compute distribution
        if fcounter == 2 # If MF computation, use previous distribution
             @load "r0/r0_N_$(N)_r_$i.jdl2" r0 L
        else
            L = (N/n)^(1/3) # Change the volume to keep the density cste
            r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]

            # Choose a distribution where the minimum distance between the atoms is bigger than d0_min
            while min_r0(r0) < d0_lb
                r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]
            end
            # Save the atoms position for comparison with QuantumOptics
            @save "r0/r0_N_$(N)_r_$i.jdl2" r0 L
        end

        # Compute the parameters
        system = SpinCollection(r0, e, gammas=1.)
        Ω_CS = OmegaMatrix(system)
        Γ_CS = GammaMatrix(system)
        Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
        Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
        exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
        conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
        p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])

         # Load the functions
        fsolve(du, u, p, t) = functions[fcounter](du, u, p0)

        phi_array_0, theta_array_0 = zeros(N), ones(N)*π # We start from all the atoms in the GS
        u0 = u0_CFunction(phi_array_0, theta_array_0, op_list, fcounter)

        prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (T[1], T[end]))
        sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5(), saveat=T;
                        reltol=1.0e-6,
                        abstol=1.0e-8)
        
        println(SciMLBase.successful_retcode(sol))
        push!(t_sol, sol.t)
        push!(popup_t, [sum(real(sol.u[i][1:N])) for i=1:length(t_sol[end])])
        push!(sol_t, sol.u)
    end
    return popup_t, sol_t, t_sol
end

""" Return the minimum distance of a distribution of atoms r0 """
function min_r0(r0)
    N = length(r0)
    d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
    for j in 1:N
        for k = 1:N
            d0[j, k] = norm(r0[j]-r0[k])
        end
    end
    return minimum(d0[d0 .> 0])
end

""" Function loading the block subfunction when a lot of equations are involved """
function load_f(fname::String, libpath::String)
	lib = Libdl.dlopen(libpath)
	fptr = Libdl.dlsym(lib, fname)
	return (du, u, params) -> ccall(fptr, Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}, Ptr{ComplexF64}), du, u, params)
end

### Define the system

In [ ]:
# Nbr of particles
N = 4
r = 10 # Nbr of repetitions

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Fixed density
n0 = 2e3 # atoms per unit of volume (already normalized)
d0_lb = 2e-10 # Minimum distance between the atoms (lower boundary) in m

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
Ω_RO = Ω_RO / γ
d0_lb = d0_lb/λ

# Quantization axis along z
e = [0, 0, 1.]

# Integration parameter
tstep = 1e-4
T = [0:tstep:100;]; # Normalised time

### Compute stationnary state for r repetitions

In [ ]:
# Create the directories
if !isdir("r0")
    mkdir("r0")
end
if !isdir("Images_distribution")
    mkdir("Images_distribution")
end
if !isdir("solutions")
    mkdir("solutions")
end
nothing 

# MPC

In [ ]:
# Prepare the wrapper
const N_FUNCS = 2  # Total function nbr
const functions = Vector{Function}(undef, N_FUNCS)

functions[1] = load_f("diffeqf", "libs/liballfuncs_$N.dll")
functions[2] = load_f("diffeqf", "libs/liballfuncs_$(N)_meanfield.dll");

In [ ]:
@load "op_list/op_list_$N.jdl2" op_list
list_r = 1:r

# Solve with Euler
chunks = Iterators.partition(list_r, cld(length(list_r), Threads.nthreads()))
tasks = map(chunks) do chunk # Split the different distributions into chuncks solved on each core
    Threads.@spawn solve_random_distrib(chunk, op_list, N, n0, d0_lb, 1) # MPC: fcounter=1
end

# Gather the data from the different threads
sol_tasks = fetch.(tasks)
popup_t_MPC, sol_t_MPC, t_MPC = vcat([s[1] for s in sol_tasks]...), vcat([s[2] for s in sol_tasks]...), vcat([s[3] for s in sol_tasks]...);

In [ ]:
close("all")
fig, ax = subplots()
for i in 1:length(popup_t_MPC)
    line, = ax.plot(t_MPC[i], popup_t_MPC[i])
end
ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")
ax.set_ylim(0, 0.1)

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

# MF

In [ ]:
@load "op_list/op_list_$(N)_meanfield.jdl2" op_list
list_r = 1:r

# Solve with Euler
chunks = Iterators.partition(list_r, cld(length(list_r), Threads.nthreads()))
tasks = map(chunks) do chunk # Split the different distributions into chuncks solved on each core
    Threads.@spawn solve_random_distrib(chunk, op_list, N, n0, d0_lb, 2) # MF: fcounter=2
end

# Gather the data from the different threads
sol_tasks = fetch.(tasks)
popup_t_MF, sol_t_MF, t_MF = vcat([s[1] for s in sol_tasks]...), vcat([s[2] for s in sol_tasks]...), vcat([s[3] for s in sol_tasks]...);

In [ ]:
close("all")
fig, ax = subplots()
for i in 1:length(popup_t_MF)
    line, = ax.plot(t_MF[i], popup_t_MF[i])
end
ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")
ax.set_ylim(0, 0.1)

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

# Compare MF/MPC

In [ ]:
close("all")
fig, ax = subplots()
for i in 1:length(popup_t_MF)
    line, = ax.plot(t_MPC[i], popup_t_MPC[i], label="MPC")
    line_MF, = ax.plot(t_MF[i], popup_t_MF[i], linestyle="--", color=line.get_color(), label="MF")
end
ax.set_xlabel(L"$\gamma t$")
ax.set_ylabel(L"$\langle  n_{\uparrow} \rangle $")
ax.legend()
ax.set_ylim(0, 0.1)

suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

# Brouillons

In [ ]:

# """ Create a random distribution, save it, computes the corresponding parameters an return the stationnary state. 
# If compute_t_evolution, compute the whole evolution, else only the stationnary state. """
# function solve_random_distrib_MF(chunk, f_MF, op_list, N, n, d0_lb)
#     popup_t, sol_t, t_euler = [], [], []

#     for i in chunk
#         # Load atom distribution of MPC
#         @load "r0/r0_N_$(N)_r_$i.jdl2" r0 L

#         # Compute the parameters
#         system = SpinCollection(r0, e, gammas=1.)
#         Ω_CS = OmegaMatrix(system)
#         Γ_CS = GammaMatrix(system)
#         Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
#         Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
#         exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
#         conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
#         p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])

#          # Load the functions
#         fsolve(du, u, p, t) = f_MF(du, u, p0)

#         phi_array_0, theta_array_0 = zeros(N), ones(N)*π # We start from all the atoms in the GS
#         u0 = u0_CFunction_MF(phi_array_0, theta_array_0, op_list)

#         prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (T[1], T[end]))
#         sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5(), saveat=T;
#                         reltol=1.0e-6,
#                         abstol=1.0e-8)
        
#         print(SciMLBase.successful_retcode(sol))
#         push!(t_euler, sol.t)
#         push!(popup_t, [sum(real(sol.u[i][1:N])) for i=1:length(t_euler[end])])
#         push!(sol_t, sol.u)
#     end
#     return popup_t, sol_t, t_euler
# end



# """ Return the minimum distance of a distribution of atoms r0 """
# function avg_r0(r0)
#     N = length(r0)
#     d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
#     for j in 1:N
#         for k = 1:N
#             d0[j, k] = norm(r0[j]-r0[k])
#         end
#     end
#     return mean(d0)*N/(N-1)
# end

# """ Reconstruct the position of atoms """
# function reconstruct_img_distrib(N, i)
#     @load "r0/r0_N_$(N)_r_$i.jdl2" r0 L
#     plt.close("all")
#     fig = plt.figure()
#     ax = fig.add_subplot(projection="3d")
#     ax.scatter([r[1] for r in r0], [r[2] for r in r0], [r[3] for r in r0])
#     ax.set_xlabel(L"x/$\lambda$")
#     ax.set_ylabel(L"y/$\lambda$")
#     ax.set_zlabel(L"z/$\lambda$")
#     ax.set_xlim(0, L), ax.set_ylim(0, L), ax.set_zlim(0, L)
#     plt.savefig("Images_distribution/IImages_distribution_N_$(N)_r_$i")
# end